<h1>Compositing Manning's n values for HEC-RAS Cross Sections</h1>
The code presented in this notebook is meant to be used in tandem with a <a href="https://www.hec.usace.army.mil/confluence/rasdocs/rasum/6.0">HEC-RAS</a> 1D model. Once the Manning's n grid from <a href="ProcessRasters.ipynb">ProcessRasters.ipynb</a> has been imported into HEC-RAS, it is possible to use HEC-RAS Geometry Editor to extract the Manning's n values into the cross section. For almost any case, this will result in too many values for HEC-RAS (at the time of writing, the limit is 20 values per cross section). This notebook shows how to use K-Means clustering to composite the Manning's n values for each cross section, and export those composites into a new HEC-RAS model.

***Make sure to back up of your model before inputting it into this code***

HEC-RAS saves Cross Section data in text files, which can be manipulated to modify Manning's n within the model. The book <a href="https://www.google.com/books/edition/Breaking_the_HEC_RAS_Code/eY7AoQEACAAJ?hl=en">Breaking the HEC-RAS Code</a> by Chris Goodell covers the structure of the geometry text files quite extensively. 

For our purposes, Manning's n values are stored as a series of 8-character value pairs: a horizontal river station, and a Manning's n value. There's 9 of these 8-character values in a line of text within the geometry file. Supplementing the text geometry file, there's an HDF file that allows us to read some further properties of each cross section. The general structure of this code is as follows:
<ol>
<li>Create a copy of the project to make sure we don't overwrite the original model (<b><u>still, please make a backup</b></u>)</li>
<li>Read the Manning's n and cross section properties from the geometry HDF file.</li>
<li>Calculate the composite values for each cross section based on K-Means clustering</li>
<li>Overwrite the copied geometry text file.</li>
</ol>

In [1]:
import os, h5py, warnings, shutil
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# SCIPY AND SCIKIT-LEARN IMPORTS
from scipy.interpolate import NearestNDInterpolator
from sklearn.cluster import KMeans # DBSCAN, MeanShift, OPTICS
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# SELF IMPORTS
from rasUtils import *

# IGNORE SKLEARN KMEANS AND NATURAL NAME WARNINGS
warnings.filterwarnings('ignore') 

c:\Users\franc\miniforge3\envs\lidarmanning\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Locations of input and output files. Indicate where the input HEC-RAS project is, along with the geomety file for modification, and directory for new HEC-RAS project

In [2]:
# INPUT HEC-RAS PROJECT DIRECTORY
ras_base_path = r"D:\Desktop\Roughness\hecras\000_V2\NLCD\\"

# OUTPUT HEC-RAS PROJECT DIRECTORY
ras_out_path  = r"D:\Desktop\Roughness\hecras\000_V2\NLCD_comp2\\"

# FILENAME OF TEXT GEOMETRY FILE
geometry_file = r"D100-00-00.g01"

# PATH TO TEXT GEOMETRY FILE
ras_geo_path  = os.path.join(ras_out_path, geometry_file)

# PATH TO HDF GEOMETRY FILE
file_path     = os.path.join(ras_base_path, geometry_file + ".hdf")

Parameters for clustering algorithm.

In [3]:
z_scale          = 2     # SCALING FACTOR FOR Z-AXIS DURING CLUSTERING
plot             = True  # DO WE EXPORT PLOTS FOR EACH XS?
use_mannings     = False # DO NOT USE. NOT IMPLEMENTED
min_num_clusters = 3     # THE MINIMUM NUMBER OF MANNING'S ROUGHNESSES FOR EACH CROSS SECTION
mann_read_sp     = 800   # SPAN FOR MANNINGS REGION. DETERMINES HOW OFTEN WE CLUSTER PER XS (num_clusters = xs_len // mann_read_sp + 1). 
                         # SET TO NONE TO USE MINIMUM FOR ALL XS.

Read geometry attributes from HDF file

In [4]:
file = h5py.File(file_path)
file_attributes = pd.read_hdf(file_path, "Geometry/Cross Sections/Attributes")

elev_info = file["Geometry"]["Cross Sections"]["Station Elevation Info"][:]
elev      = file["Geometry"]["Cross Sections"]["Station Elevation Values"][:]
mann_info = file["Geometry"]["Cross Sections"]["Manning's n Info"][:]
mann      = file["Geometry"]["Cross Sections"]["Manning's n Values"][:]

Create destination folder for output HEC-RAS project

In [5]:
def copy_folder(src_folder, dst_folder):
    # COPIES FOLDER, DELETING DESTINATION FOLDER IF NEEDED
    if os.path.exists(dst_folder):
        shutil.rmtree(dst_folder)
    shutil.copytree(src_folder, dst_folder)

copy_folder(ras_base_path, ras_out_path)

with open(ras_geo_path, 'r') as file:
    lines = file.readlines()

Bulk of processing occurs here - cluster each cross section, and get text lines to add in geometry text file.

In [6]:
if plot: 
    # IF WE'RE PLOTTING, CREATE OUTPUT DIRECTORY FOR PLOTS IN OUTPUT HEC-RAS PROJECT FOLDER
    plot_path = os.path.join(ras_out_path, "xs_plots/")
    os.makedirs(plot_path, exist_ok=True)

scaler     = StandardScaler()

values  = []            # DEBUGGING STRUCTURE TO KEEP TRACK OF SORTED MANNING'S N VALUES FOR EACH CROSS SECTION
output_text_lines = []  # TEXT LINES OBTAINED FOR EACH CROSS SECTION

# FOR EACH CROSS SECTION
for idx in tqdm(range(elev_info.shape[0])):

    elevations = getElevations(idx, elev_info, elev) # GET HORIZONTAL STATION ELEVATION
    mannings   = getMannings(idx, mann_info, mann)   # GET MANNING'S N FOR EACH HORIZONTAL STATION
    
    xs_len = np.max(elevations[:, 0]) # CROSS SECTION LENGTH

    if mann_read_sp is not None:
        # CALCULATE HOW MANY CLUSTERS WE NEED
        num_clusters = int(xs_len // mann_read_sp + 1)
    
        # ENFORCE MINIMUM NUMBER OF CLUSTERS
        if num_clusters < min_num_clusters:
            num_clusters = min_num_clusters
    
        # ENFORCE HEC-RAS MAX NUMBER OF CLUSTERS
        if num_clusters > 19:
            num_clusters = 19
    else:
        num_clusters = min_num_clusters

    # CHECK IF WE ACTUALLY NEED TO SUMMARIZE ANYTHING (SKIP IF THERE'S FEWER MANNING'S N THAN OUTPUT CLUSTERS)
    if mannings.shape[0] <= num_clusters:
        print(f"{title(idx, file_attributes)} only has {mannings.shape[0]} Manning's n values, determined {num_clusters} clusters with {xs_len:.2f} XS length. Skipping.")
        text_line = create_Manning_lines(mannings[:, 0].round(3).tolist(), mannings[:, 1].tolist())
        output_text_lines.append(text_line)
        continue
    
    clustering = KMeans(n_clusters=num_clusters)
    
    # INTERPOLATION TO FILL ALL CROSS SECTION COORDINATES WITH MANNING'S N
    interpolator = NearestNDInterpolator(transform(mannings[:, 0]), mannings[:, 1])
    elev_mannings = interpolator(transform(elevations[:, 0]))

    if use_mannings:

        raise(Exception("Not implemented! Currently, this may result in more XS regions than allowed."))
        
        # NOT IMPLEMENTED BECAUSE IT MAY LEAD TO MORE XS MANNINGS REGIONS
        # THAN ALLOWED. IT'S DIFFICULT TO CONTROL HOW MANY CONTIGUOUS REGIONS ARE CREATED IF CONSIDERING THIS
        # RECOMMENDED STRATEGY IS TO CREATE TONS OF REGIONS AND THEN GROUP SIMILAR NEIGHBORS.
        
        # CLUSTER BASED ON X-Z LOCATIONS AND MANNINGS COEFFICIENTS
        # SCALE COMPONENTS (DIFFERENT UNITS!)
        datastruct  = np.vstack((elevations.T, elev_mannings.T)).T
        scaled_data = scaler.fit_transform(datastruct)

        # CLUSTER
        clustering.fit(scaled_data)
        labels = clustering.labels_
    else:
        # CLUSTER BASED ON X-Z LOCATIONS
        datastruct = elevations.copy()
        datastruct[:, 1] = datastruct[:, 1] * z_scale
        clustering.fit(datastruct)
        labels = clustering.labels_
    
    # CALCULATE WETTED PERIMETER
    perimeters = calcWettedPerimeter(elevations)
    
    region_mann_n  = []
    region_station = []

    if plot:
        plt.figure(figsize=(8, 6))
    
    # FOR EACH LABEL GET RIVER STATION AND COMPOSE MANNING'S
    for label in np.unique(labels): # FOR EACH CLUSTER
        mannings    = elev_mannings[labels == label] # GET MANNING'S N
        curr_coords = elevations[labels == label]    # GET RIVER STATION ELEVATION
        curr_pers   = perimeters[labels == label]    # GET WETTED PERIMETERS
        
        # COMPOSE MANNING'S N AND APPEND TO OUTPUT LIST 
        region_mann_n.append(composeManningsN(curr_pers, mannings))

        # MINIMUM X-MEASUREMENT FOR RIVER STATION IN CLUSTER 
        # (HEC-RAS ASSIGNS THE SAME VALUE TO ANY SUBSEQUENT RIVER STATIONS WITH BLANK MANNING'S N)
        region_station.append(np.min(curr_coords[:, 0]))

        if plot:
            plt.scatter(curr_coords[:,0], curr_coords[:, 1], label=f'Cluster {label}', 
                        s=10, color=np.random.rand(3,))
            plt.text(curr_coords[:,0].mean(), curr_coords[:,1].mean(), f"{region_mann_n[-1]:.3f}")
    if plot:
        mytitle = title(idx, file_attributes)
        plt.title(mytitle + f" Clusters: {num_clusters}")
        plt.xlabel("XS Station")
        plt.ylabel("Elevation")
        plt.savefig(os.path.join(plot_path, mytitle.replace(":", "-").replace(" ", "_")))
        plt.close()

    # NOW THAT WE HAVE THE MINIMUM CLUSTER RIVER STATION, ALONG WITH THEIR RESPECTIVE MANNING'S N, WE NEED TO ORDER THEM FROM LEAST TO GREATEST
    # SORT LEAST TO GREATEST XS STATION
    sorted_list = sorted(zip(region_station, region_mann_n))

    # EXTEND DEBUGGING OUTPUT STRUCTURE 
    values.extend(sorted_list)
    
    # GET TEXT LINES TO REPLACE IN TEXT GEOMETRY FILE FOR THIS CROSS SECTION
    text_line = create_Manning_lines(region_station, region_mann_n)
    output_text_lines.append(text_line)

  0%|          | 0/387 [00:00<?, ?it/s]

River: D100-00-00 RS: 157812 only has 3 Manning's n values, determined 3 clusters with 640.96 XS length. Skipping.
River: D100-00-00 RS: 157586 only has 2 Manning's n values, determined 3 clusters with 640.96 XS length. Skipping.


Find where we need to replace the lines in text geometry file (```"#Mann="``` is the start and ```"#Bank Sta="``` is the end)

In [7]:
def get_matching_line_indices(lines, starting_string):
    return [idx for idx, line in enumerate(lines) if line.startswith(starting_string)]

# FIND THE INDICES OF LINES TO REPLACE MANNING'S N
start_lines = get_matching_line_indices(lines, "#Mann=")
end_lines   = get_matching_line_indices(lines, "Bank Sta=")

Save text geometry file based on outputs

In [8]:
# REVERSE ORDER ALLOWS US TO USE THE INDICES WE FOUND BEFORE (IF WE WENT FORWARD, WE WOULD CHANGE THE LINE INDICES)
for i, start_line in reversed(list(enumerate(start_lines))):
    end_line = end_lines[i]
    
    # DELETE LINES 
    del lines[start_line:end_line]
    
    # INSERT LINES
    lines[start_line:start_line] = output_text_lines[i]
    
# WRITE FILE
with open(ras_geo_path, 'a') as file:
    file.truncate(0)
    file.writelines(lines)